In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from collections import OrderedDict

In [17]:
HYPEROPTPATH = Path("/home/michaeldodds/Projects/manchester/computer_vision/results/hyperopt/")
RESULTPATH = HYPEROPTPATH / "final_iterative_exps.csv"
RESULTPATH_CNN = HYPEROPTPATH / "cnn_iterative_experiments2.csv"
df = pd.read_csv(RESULTPATH)
df_cnn = pd.read_csv(RESULTPATH_CNN)

In [18]:
df_cnn.columns.value_counts()

params             1
mean_test_score    1
std_test_score     1
dataset            1
norm_type          1
colour             1
classifier         1
Name: count, dtype: int64

In [19]:
df_cnn

,params,mean_test_score,std_test_score,dataset,norm_type,colour,classifier
0,"OrderedDict({'batch_size': 4, 'epochs': 30, 'n...",0.458358,0.024452,mammals,32,rgb,split_cnn
1,"OrderedDict({'batch_size': 4, 'epochs': 30, 'n...",0.457355,0.013483,mammals,32,rgb,split_cnn
2,"OrderedDict({'batch_size': 4, 'epochs': 27, 'n...",0.446518,0.011532,mammals,32,rgb,split_cnn
3,"OrderedDict({'batch_size': 4, 'epochs': 24, 'n...",0.441501,0.024063,mammals,32,rgb,split_cnn
4,"OrderedDict({'batch_size': 4, 'epochs': 30, 'n...",0.434076,0.018828,mammals,32,rgb,split_cnn
...,...,...,...,...,...,...,...
91,"OrderedDict({'batch_size': 4, 'epochs': 9, 'ne...",0.295206,0.034559,stl10,32,greyscale,classic_cnn
92,"OrderedDict({'batch_size': 8, 'epochs': 17, 'n...",0.261007,0.026571,stl10,32,greyscale,classic_cnn
93,"OrderedDict({'batch_size': 16, 'epochs': 23, '...",0.255602,0.027800,stl10,32,greyscale,classic_cnn
94,"OrderedDict({'batch_size': 8, 'epochs': 14, 'n...",0.251594,0.030819,stl10,32,greyscale,classic_cnn


In [20]:
df_cnn['colour'] = df_cnn['colour'].fillna(3)
cols = ['dataset', 'norm_type', 'colour', 'classifier']
idx = df_cnn.groupby(cols)['mean_test_score'].idxmax()
cnn_aggregated_results = df_cnn.groupby(cols).aggregate({'mean_test_score':['max']})
best_rows = df_cnn.loc[idx].sort_values('mean_test_score', ascending=False)
cnn_aggregated_results

mean_test_score
                                                    max
dataset norm_type colour    classifier                 
mammals 32        greyscale classic_cnn        0.169376
                  rgb       classic_cnn        0.256271
                            split_cnn          0.458358
stl10   32        greyscale classic_cnn        0.408796
                  rgb       classic_cnn        0.432998
                            split_cnn          0.581203

In [21]:
params = list(best_rows['params'])
param_dic = {i: eval(parms, {'OrderedDict' : OrderedDict}) for i, parms in enumerate(params)}
param_frame = pd.DataFrame(param_dic).T
param_frame.columns = [name.split("__", 1)[-1] for name in param_frame.columns.values]
param_frame.index = best_rows.index
best_rows_with_params = pd.concat([best_rows, param_frame], axis=1)

In [22]:
cols = ['dataset', 'classifier', 'keypoints_algo', 'descriptor_algo']
idx = df.groupby(cols)['mean_test_score'].idxmax()
aggregated_results = df.groupby(cols).aggregate({'mean_test_score':['max']})
best_rows = df.loc[idx].sort_values('mean_test_score', ascending=False)
aggregated_results

mean_test_score
                                                              max
dataset classifier keypoints_algo descriptor_algo                
mammals knn        dense          dense_hist             0.202993
                                  sift                   0.171493
                   none           none                   0.135997
                   sift           dense_hist             0.185987
                                  sift                   0.147499
        rf         dense          dense_hist             0.219991
                                  sift                   0.188001
                   none           none                   0.196002
                   sift           dense_hist             0.201488
                                  sift                   0.172998
stl10   knn        dense          dense_hist             0.288043
                                  sift                   0.353003
                   none           none                   0.279966
                   sift           dense_hist             0.241987
                                  sift                   0.265014
        rf         dense          dense_hist             0.339004
                                  sift                   0.409032
                   none           none                   0.338000
                   sift           dense_hist             0.292020
                                  sift                   0.314018

In [23]:
aggregated_results_outpath = Path(HYPEROPTPATH, 'aggregated_results')
cv_agg_results_outpath = aggregated_results_outpath / 'cv_aggregated_results.csv'
cv_results_outpath = aggregated_results_outpath / 'cv_best_classifiers.csv'

cnn_agg_results_outpath = aggregated_results_outpath / 'cnn_aggregated_results.csv'
cnn_results_outpath = aggregated_results_outpath / 'cnn_best_classifiers.csv'

aggregated_results_outpath.mkdir(parents=True, exist_ok=True)

aggregated_results.to_csv(cv_results_outpath, index=True)
best_rows.to_csv(cv_results_outpath, index=False)

cnn_aggregated_results.to_csv(cnn_agg_results_outpath, index=True)
best_rows_with_params.to_csv(cnn_results_outpath, index=False)

In [193]:
zero = best_rows_with_params.iloc[0]
one = best_rows_with_params.iloc[1]

In [199]:
pd.concat([zero, one], axis=1).T

,params,mean_test_score,std_test_score,dataset,norm_type,colour,classifier,batch_size,epochs,channels,class_heads,kernel_width,lr,momentum
64,"OrderedDict({'batch_size': 4, 'epochs': 20, 'n...",0.435399,0.006869,stl10,32,3.0,cnn,4.0,20.0,3.0,10.0,5.0,0.001,0.99
48,"OrderedDict({'batch_size': 4, 'epochs': 20, 'n...",0.426198,0.014646,stl10,eq_hist,3.0,cnn,4.0,20.0,3.0,10.0,5.0,0.1,0.8
